In [1]:
!nvidia-smi

Fri Mar 15 15:55:23 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:01:00.0 Off |                  N/A |
| 23%   33C    P8    17W / 250W |      0MiB / 12195MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from os import path as p
import os
from plot_history import plot_history

In [3]:
from keras.layers import Input
from keras.layers.core import Dense, Dropout
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Flatten
from keras.models import Model
from keras.losses import mse
from keras.optimizers import Adadelta, Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow.keras.backend as K

Using TensorFlow backend.


In [4]:
data_dir = '/storage/yw18581/data/'
data_folder = os.path.join(data_dir, 'train_validation_test')

In [5]:
X_train = np.load(os.path.join(data_folder, 'Xy_train_dist.npz'))["y"]
y_train = np.load(os.path.join(data_folder, 'Xy_train_dist.npz'))["dist"]

In [6]:
X_val = np.load(os.path.join(data_folder,'Xy_val_dist.npz'))["y"]
y_val = np.load(os.path.join(data_folder, 'Xy_val_dist.npz'))["dist"]

In [7]:
X_test = np.load(os.path.join(data_folder, 'Xy_test_dist.npz'))["y"]
y_test = np.load(os.path.join(data_folder, 'Xy_test_dist.npz'))["dist"]

In [8]:
X_pred = np.load(os.path.join(data_dir, 'trained_models/UNet_training_generator_1500epochs/Xy_test_predicted_UNet.npz'))['y']
y_pred = np.load(os.path.join(data_dir, 'trained_models/UNet_training_generator_1500epochs/Xy_test_predicted_UNet.npz'))['dist']

In [9]:
def cut_X(arr, reshape = None):
    x_cut = arr[:,960:1300,600:]
    if reshape:
        if len(x_cut.shape)>3:
            x_cut = x_cut[...,0]
            x_cut_out = x_cut.reshape(x_cut.shape[0],x_cut.shape[1]*x_cut.shape[2])
    else:
        x_cut_out = x_cut
    return x_cut_out

In [10]:
X_train_cut = cut_X(X_train)

In [11]:
X_val_cut = cut_X(X_val)

In [12]:
X_test_cut = cut_X(X_test)

In [13]:
X_pred_cut = cut_X(X_pred)

In [14]:
N_EPOCHS = 200
BATCH_SIZE = 16

CHECKPOINT_FOLDER_PATH = p.join(data_dir, 'trained_models')
TASK_NAME = 'Leaf_position_regression'
TASK_FOLDER_PATH = p.join(CHECKPOINT_FOLDER_PATH, TASK_NAME)

if not p.exists(TASK_FOLDER_PATH):
    os.makedirs(TASK_FOLDER_PATH)

In [15]:
K.clear_session()

In [16]:
def create_model(input_shape):
    input_layer = Input(shape=input_shape,name='input')
    x = Conv2D(64,kernel_size=3, activation='relu')(input_layer)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(128, activation ='sigmoid')(x)
    prediction = Dense(1, activation='linear')(x)
    model = Model(inputs = input_layer, outputs = prediction)
    return model

model = create_model(input_shape=(X_train_cut.shape[1:]))
OPTIMIZER = 'Adam'
model.compile(loss='mse',
              optimizer=Adam(), metrics = ['mse'])
callbacks = [EarlyStopping(monitor='val_loss',patience = 8, verbose = True), 
             ModelCheckpoint(os.path.join(TASK_FOLDER_PATH,"leaf_regression_CNN_short_relu_{}epochs_{}batch_{}.hdf5".format(N_EPOCHS,BATCH_SIZE, OPTIMIZER)), 
                             monitor='val_loss')]
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 340, 800, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 338, 798, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 169, 399, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4315584)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               552394880 
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 552,395,649
Trainable params: 552,395,649
Non-trainable params: 0
______________________________________________________________

In [17]:
history = model.fit(X_train_cut, y_train, validation_data=(X_val_cut, y_val),
                    epochs=N_EPOCHS, verbose=True, batch_size=BATCH_SIZE, shuffle=True, 
                    callbacks = callbacks)

Train on 768 samples, validate on 192 samples
Epoch 1/200


ResourceExhaustedError: OOM when allocating tensor with shape[4315584,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/mul_13}} = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Adam/beta_2/read, training/Adam/Variable_8/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
plot_history(history)

model.save(os.path.join(data_dir,'trained_models','leaf_regression_shallow.hdf5'))

preds = model.predict(X_test_cut)

preds.ravel().shape

y_test.shape

from sklearn.metrics import mean_squared_error as mse

mse(y_test, preds.ravel())

np.unique(y_test)

t_test_2 = y_test[y_test==2]
t_test_4 = y_test[y_test==4]
t_test_10 = y_test[y_test==10]
t_test_25 = y_test[y_test==25]

print(t_test_2.shape, t_test_4.shape, t_test_10.shape, t_test_25.shape)

preds = preds.ravel()

preds

p2 = preds[y_test==2]
p4 = preds[y_test==4]
p10 = preds[y_test==10]
p25 = preds[y_test==25]

mse(t_test_2, p2)

mse(t_test_4, p4)

mse(t_test_10, p10)

mse(t_test_25, p25)

from matplotlib.image import imread

w = os.walk(os.path.join(data_dir, "10x10_15mm_v2_8bit_test"))
folder, _, files = next(w)

folder

files

gt_15 = [os.path.join(folder, f) for f in files]

gt_15

X_15 = [imread(fp)[ROW_SLICE,COL_SLICE] for fp in gt_15]

IMG_WIDTH = 1400
IMG_HEIGHT = 1400
ROW_SLICE = slice(0, 1400)
COL_SLICE = slice(1000, None)


X_15_cut = cut_X(np.asarray(X_15))

preds15 = model.predict(X_15_cut)

preds15